In [ ]:
import numpy as np
import os
import pandas as pd
import sys
import torch

from matplotlib import pyplot as plt
from tqdm import tqdm

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [ ]:
x = torch.arange(9).reshape(3,3)
idx = torch.tril_indices(3, 3, offset=-1)

# Usa los índices para obtener los elementos de la parte inferior triangular
lower_triangular_elements = x[idx[0], idx[1]]
x[idx[0], idx[1]] = -2
print(lower_triangular_elements)
print(x)

In [ ]:
argwhere_format = torch.stack(idx, dim=1)
argwhere_format

In [ ]:
import torch

# Genera los índices de la parte inferior triangular de una matriz de 3x3
indices = torch.tril_indices(row=3, col=3, offset=0)

# Transponer y apilar los índices para obtener el formato similar a argwhere
argwhere_format = torch.stack((indices[0], indices[1]), dim=1)
print(argwhere_format)

In [ ]:
x, y = zip(indices)
print(x, y)

In [ ]:
import torch

from VAE.nn.VariationalLayer import VariationalLayer
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch import nn
import numpy as np

class AnomalyDetector(nn.Linear, VariationalLayer):
    '''
        This implementation is for anomaly detection.
    '''
    def __init__(self, in_features, out_features, bias=True, full_cov=False, sigma_anomaly=3) -> None:
        assert out_features > 1, "The Gaussian posterior variables must be greater than 1"

        super(AnomalyDetector, self).__init__(in_features, out_features, bias)
        self.log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
        self.log_sigma_bias = Parameter(torch.Tensor(out_features))

        if full_cov:
            # Correlation estimation
            self.corr_weight = Parameter(torch.Tensor(out_features, in_features))
            self.corr_bias = Parameter(torch.Tensor(out_features))
        
        torch.nn.init.xavier_uniform_(self.log_sigma_weight)
        self.log_sigma_bias.data.fill_(2*np.log((2*sigma_anomaly)))
        
        self.sigma_anomaly = sigma_anomaly
        self.mu, self.sigma = None, None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mu = F.linear(x, self.weight, self.bias) 
        sigma = torch.exp(0.5 * F.linear(x, self.log_sigma_weight, self.log_sigma_bias))

        # if self.training:
        self.mu, self.sigma = mu, sigma

        # Reparameterization trick
        eps = torch.normal(0, torch.ones_like(sigma))
        return mu + sigma * eps

    def kl_reg(self, targets: torch.Tensor) -> torch.Tensor:
        # KL-Divergence regularization
        assert torch.all((targets == 0) | (targets == 1))
        sigma_2 = (torch.ones_like(targets) + ((self.sigma_anomaly-1) * targets)).unsqueeze(1).cuda()
        result = (torch.log(sigma_2) - torch.log(self.sigma) + (self.sigma**2 + self.mu**2)/(2*sigma_2**2) - 0.5)
        
        return result.mean() 

## Correlation and Covariance Implementation

In [ ]:
import math
in_features, out_features = 10, 3
corr_size = math.comb(out_features, 2)
corr_weight = Parameter(torch.Tensor(corr_size, in_features))
corr_bias = Parameter(torch.Tensor(corr_size))

log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
log_sigma_bias = Parameter(torch.Tensor(out_features))

torch.nn.init.xavier_uniform_(log_sigma_weight)
log_sigma_bias.data.fill_(2*np.log((2*1)))

torch.nn.init.xavier_uniform_(corr_weight)
corr_bias.data.fill_(0)

x = torch.rand((5, 10))

In [ ]:
corr = F.tanh(F.linear(x, corr_weight, corr_bias))
sigma = torch.exp(0.5 * F.linear(x, log_sigma_weight, log_sigma_bias))

print(corr)
print(sigma)

In [ ]:
cov1 = corr[:, 0] * (sigma[:, 0] * sigma[:, 1])
cov2 = corr[:, 1] * (sigma[:, 0] * sigma[:, 2])
cov3 = corr[:, 2] * (sigma[:, 1] * sigma[:, 2])
cov1, cov2, cov3

In [ ]:
idx = torch.triu_indices(corr.size(1), corr.size(1), offset=1)

a = torch.zeros((3,3))
a[tuple(idx)] = 1
a

In [ ]:
# idx = list(combinations(range(out_features), 2))
idx = torch.triu_indices(*corr.size(), offset=1)
idx.T

In [ ]:
corr.shape

In [ ]:
idx = torch.triu_indices(*corr.size(), offset=1)
test = sigma[:, idx.T]

cov = test.prod(dim=-1) * corr
cov.shape

In [ ]:
def correlation_tests():
    sigma = torch.ones(1,3)*2
    corr = torch.tensor([[1, .5, .25]])

    cov_1 = corr[0,0] * (sigma[0, 0] * sigma[0, 1])
    cov_2 = corr[0,1] * (sigma[0, 0] * sigma[0, 2])
    cov_3 = corr[0,2] * (sigma[0, 1] * sigma[0, 2])

    print(sigma, corr)
    print(cov_1, cov_2, cov_3)

correlation_tests()

In [ ]:
sigmas = torch.arange(0.1, 1.21, 0.1)
sigmas

In [ ]:
indices = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
print(indices)

In [ ]:
import math
math.comb(4, 2)

In [ ]:
torch.diag_embed(sigma)

In [ ]:
n_latent = 4
tri_idx_row, tri_idx_col = torch.tril_indices(row=n_latent, col=n_latent, offset=-1)

print(len(tri_idx_col))